# 클러스터링 분석


In [ ]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv(r"C:\Users\user\Desktop\real\분석용데이터_소득생활인구_아동인구_보간완료.csv", encoding='utf-8')

# 데이터 확인
print(df.head())

In [ ]:
# 데이터 확인 후 최종으로 사용할 변수 가공하기 

# 총생활인구_log (2022~2024년 합쳐 평균 후 로그)
population_cols = [col for col in df.columns if '총생활인구_' in col]
df['총생활인구_평균'] = df[population_cols].mean(axis=1)
df['총생활인구_log'] = np.log1p(df['총생활인구_평균'])

# 유아시설_수 (유치원+어린이집)
df['유아시설_수'] = df['유치원_수'] + df['어린이집_수']

# 아동인구_log (0~9세 전체 연도 평균 후 로그)
child_population_cols = [col for col in df.columns if '아동인구_' in col]
df['아동인구_평균'] = df[child_population_cols].mean(axis=1)
df['아동인구_log'] = np.log1p(df['아동인구_평균'])

# 소득_log (22~24년 전체 평균 후 로그)
income_cols = [col for col in df.columns if '소득_' in col]
df['소득_평균'] = df[income_cols].mean(axis=1)
df['소득_log'] = np.log1p(df['소득_평균'])

# 대중교통_수 (버스정류장 + 지하철역)
df['대중교통_수'] = df['버스정류장_수'] + df['지하철역_수']

# 돌봄시설_수 (키움센터 개수 그대로 사용)
df['돌봄시설_수'] = df['키움센터_개수']

# 일반키즈카페_수, 초등학교_수는 기존 그대로 사용할 거라 손대지 않음음

# 클러스터링 분석 최종 사용할 변수만 선택
final_df = df[['행정동명', '서울형키즈카페_개수', '일반키즈카페_개수', '총생활인구_log', 
               '유아시설_수', '아동인구_log', '초등학교_수', 
               '돌봄시설_수', '대중교통_수', '소득_log']]

# 컬럼명 통일
final_df.rename(columns={'일반키즈카페_개수': '일반키즈카페_수'}, inplace=True)

print(final_df.head())


In [ ]:
# 클러스터링에 들어가기 전 사용할 독립변수 표준화가 필요하다고 느껴서 만든 코드
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 클러스터링에 사용할 독립변수 표준화 (행정동명, 서울형 키즈카페 개수 제외)
X = final_df.drop(['행정동명', '서울형키즈카페_개수'], axis=1)
X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=final_df.index)

# 클러스터링 결과 확인을 위해 행정동명과 종속변수 추가
X_scaled_df['행정동명'] = final_df['행정동명']
X_scaled_df['서울형키즈카페_개수'] = final_df['서울형키즈카페_개수']

print(X_scaled_df.head())


In [ ]:
# 엘보우 분석 시각화 코드
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# 한글 깨짐 방지용 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

sse = []
X_cluster = X_scaled_df.drop(['행정동명', '서울형키즈카페_개수'], axis=1)

for k in range(1, 11):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_cluster)
    sse.append(km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), sse, marker='o')
plt.xlabel('클러스터 수', fontsize=13)
plt.ylabel('SSE (오차제곱합)', fontsize=13)
plt.title('클러스터 개수 결정 (Elbow 방법)', fontsize=15)
plt.grid()
plt.show()


In [ ]:
# 실루엣 분석 시각화 코드
from sklearn.metrics import silhouette_score


silhouette_scores = []
range_n_clusters = range(2, 11)  # 최소 2개 클러스터 이상부터 가능

X_cluster = X_scaled_df.drop(['행정동명', '서울형키즈카페_개수', '클러스터'], axis=1, errors='ignore')

for n_clusters in range_n_clusters: 
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_cluster)
    
    silhouette_avg = silhouette_score(X_cluster, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"클러스터 수: {n_clusters}, 실루엣 점수: {silhouette_avg:.4f}") # 클러스터 수, 실루엣 점수 출력
    

# 실루엣 분석 시각화
plt.figure(figsize=(8, 5))
plt.plot(range_n_clusters, silhouette_scores, marker='o')
plt.xlabel('클러스터 수', fontsize=13)
plt.ylabel('실루엣 점수 (Silhouette Score)', fontsize=13)
plt.title('실루엣 분석을 통한 클러스터 개수 결정', fontsize=15)
plt.grid()
plt.show()


In [ ]:
# K-MEANS 분석 코드
from sklearn.cluster import KMeans

# 분석에 사용할 변수 설정 (클러스터 변수 제외)
X_final = X_scaled_df.drop(['행정동명', '서울형키즈카페_개수'], axis=1)

# KMeans 클러스터링 수행 (4개)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
X_scaled_df['클러스터'] = kmeans.fit_predict(X_final)

# 결과 확인
print(X_scaled_df[['행정동명', '클러스터']].head())


In [ ]:
# 클러스터별 주요 변수 평균값 분석 (숫자형만)
cluster_summary_mean = X_scaled_df.groupby('클러스터').mean(numeric_only=True).reset_index()
print("클러스터별 평균:\n", cluster_summary_mean)

# 클러스터별 주요 변수 중앙값 분석 (숫자형만)
cluster_summary_median = X_scaled_df.groupby('클러스터').median(numeric_only=True).reset_index()
print("\n클러스터별 중앙값:\n", cluster_summary_median)

# 클러스터별 행정동 수 확인 (문제 없음)
cluster_counts = X_scaled_df['클러스터'].value_counts()
print("\n클러스터별 행정동 수:\n", cluster_counts)


In [ ]:
# 클러스터 이름 매핑
cluster_name_dict = {
    0: '기존 아동친화 지역',
    1: '교통 우수형 신규추천 지역',
    2: '소득 상위형 (공공성 부족)',
    3: '생활인구 밀집 신규추천 지역'
}

# 한글명 적용
X_scaled_df['클러스터_이름'] = X_scaled_df['클러스터'].map(cluster_name_dict)

# 확인
print(X_scaled_df[['행정동명', '클러스터', '클러스터_이름']].head())


In [ ]:
# 클러스터별 평균 계산
cluster_mean_df = X_scaled_df.groupby('클러스터_이름')[features].mean().reset_index()

# 시각화
for feature in features:
    plt.figure(figsize=(10,10))
    sns.barplot(x='클러스터_이름', y=feature, data=cluster_mean_df, palette='Set2')
    plt.title(f'클러스터별 {feature} 평균값', fontsize=25, weight='bold')
    plt.xlabel('클러스터', fontsize=15)
    plt.ylabel(f'{feature} 평균값', fontsize=15)
    plt.grid(axis='y')
    plt.xticks(rotation=0)
    plt.show()


In [ ]:
# KMEANS - PCA 시각화 과정 코드
from sklearn.decomposition import PCA

# 클러스터링에 사용된 변수만 추출
features = ['총생활인구_log', '아동인구_log', '소득_log', '대중교통_수', 
            '유아시설_수', '초등학교_수', '돌봄시설_수', '일반키즈카페_수']

X_pca_input = X_scaled_df[features]

# PCA 2차원으로 압축
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_pca_input)

# 결과를 데이터프레임에 추가
X_scaled_df['PCA1'] = pca_result[:,0]
X_scaled_df['PCA2'] = pca_result[:,1]

# 결과 확인
print(X_scaled_df[['행정동명', 'PCA1', 'PCA2', '클러스터_이름']].head())


In [ ]:
# KMEANS 클러스터링 PCA 시각화 코드 

import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 
plt.rcParams['font.family'] = 'Malgun Gothic'  
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PCA1', y='PCA2', 
    hue='클러스터_이름', 
    palette='Set2', 
    data=X_scaled_df, 
    s=100, alpha=0.8
)

plt.title('서울형 키즈카페 입지추천 클러스터링 PCA 시각화', fontsize=20, weight='bold')
plt.xlabel('PCA 주성분 1', fontsize=14)
plt.ylabel('PCA 주성분 2', fontsize=14)
plt.legend(title='클러스터 유형', fontsize=12, title_fontsize=12) 
plt.grid(True)
plt.show()


In [ ]:
# DBSCAN 분석 코드 
from sklearn.cluster import DBSCAN

# 추천 파라미터 설정 (행정동 데이터 특성상 해야함)
dbscan = DBSCAN(eps=2.0, min_samples=4)

# DBSCAN 수행 (이미 표준화된 데이터 사용함)
X_scaled_df['DBSCAN_클러스터'] = dbscan.fit_predict(X_scaled_df[features])

# 클러스터 결과 확인
print(X_scaled_df[['행정동명', 'DBSCAN_클러스터']].head(10))


In [ ]:
# 클러스터별 행정동 수 분석 (이상치 포함)
print("클러스터별 행정동 수\n", X_scaled_df['DBSCAN_클러스터'].value_counts())

# 클러스터별 주요 변수 평균
dbscan_cluster_summary = X_scaled_df.groupby('DBSCAN_클러스터')[features].mean()
print("DBSCAN 클러스터별 특성 평균\n", dbscan_cluster_summary)


In [ ]:
# 클러스터 한글명 매핑
dbscan_cluster_name_dict = {
    -1: '특이 지역 (이상치)',
    0: '일반 밀집지역 (표준지역)',
    1: '고밀도 특화지역 (밀집 특성 강함)'
}

# 적용
X_scaled_df['DBSCAN_클러스터_이름'] = X_scaled_df['DBSCAN_클러스터'].map(dbscan_cluster_name_dict)

# 결과 확인
print(X_scaled_df[['행정동명', 'DBSCAN_클러스터', 'DBSCAN_클러스터_이름']].head(10))


In [ ]:
# DBSCAN 분석 시각화 코드
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# PCA 수행 
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled_df[features])

X_scaled_df['PCA1_DBSCAN'] = pca_result[:, 0]
X_scaled_df['PCA2_DBSCAN'] = pca_result[:, 1]

# 시각화
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PCA1_DBSCAN', y='PCA2_DBSCAN',
    hue='DBSCAN_클러스터_이름',
    palette='Set2',
    data=X_scaled_df,
    s=100, alpha=0.8
)

plt.title('DBSCAN 클러스터링 PCA 시각화 (서울형 키즈카페 입지추천)', fontsize=20, weight='bold')
plt.xlabel('PCA 주성분 1', fontsize=14)
plt.ylabel('PCA 주성분 2', fontsize=14)
plt.legend(title='DBSCAN 클러스터 유형', fontsize=11, title_fontsize=12)
plt.grid(True)
plt.show()


# KMEANS + DBSCAN 분석 합쳐서 입지 추천

In [ ]:
import pandas as pd

# 비교표 생성 (행정동명, K-Means 클러스터, DBSCAN 클러스터)
comparison_df = X_scaled_df[['행정동명', '클러스터_이름', 'DBSCAN_클러스터_이름']]

# 두 클러스터링 결과 상위 20개만 확인 예시
print(comparison_df.head(20))

# 클러스터링 결과 병합
df_merged = pd.merge(comparison_df, 
                     X_scaled_df[['행정동명', '클러스터_이름']], 
                     on='행정동명', how='left')

df_merged = pd.merge(df_merged, 
                     comparison_df[['행정동명', 'DBSCAN_클러스터_이름']], 
                     on='행정동명', how='left')

# 병합된 결과 확인
print(df_merged.head())

In [ ]:
# 클러스터링 결과를 원본 데이터와 병합하기 위해 원본 데이터 로드
import pandas as pd
df_original = pd.read_csv("C:/Users/user/Desktop/real/분석용데이터_소득생활인구_아동인구_보간완료.csv", encoding='utf-8')

# 데이터 확인
print(df_original.columns)

# comparison_df는 이미 존재하므로 그대로 사용
df_merged = pd.merge(comparison_df, df_original, on='행정동명', how='left')

# 병합 결과 확인
print(df_merged.head())

In [ ]:
# 클러스터링 결과를 명확히 담고 있는 DataFrame 생성
comparison_df = X_scaled_df[['행정동명', '클러스터_이름', 'DBSCAN_클러스터_이름']]

# 중복제거
comparison_df = comparison_df.drop_duplicates(subset=['행정동명'])

# 병합 확인
print(comparison_df.head(20))

In [ ]:
# 최적 후보군 설정 KMEANS + DBSCAN
final_recommendation_public = comparison_df[
    (
        # 생활인구가 많거나 교통 접근성 우수
        (comparison_df['클러스터_이름'].isin(['생활인구 밀집 신규추천 지역', '교통 우수형 신규추천 지역'])) &
        # 동시에 소득 수준은 중간 이하 (공공성 확보 필요)
        (comparison_df['DBSCAN_클러스터_이름'] == '일반 밀집지역 (표준지역)')
    ) |
    (
        # 생활인구 매우 많음 (밀집 특성 강함), 공공성 지역
        (comparison_df['DBSCAN_클러스터_이름'] == '고밀도 특화지역 (밀집 특성 강함)') &
        (comparison_df['클러스터_이름'] != '소득 상위형 (공공성 부족)')
    )
].copy()

# 추천 후보 행정동 리스트 출력
print("서울형 키즈카페 KMEANS + DBSCAN 요건 반영 최종 후보 행정동")
print(final_recommendation_public)


# MCDA 점수 계산

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv(r"C:\Users\user\Desktop\real\분석용데이터_소득생활인구_아동인구_보간완료.csv", encoding='utf-8')

# 필요한 컬럼 새롭게 생성 

# 아동인구 평균 (0~9세 전체 합, 22년~24년 모든 분기 평균)
child_cols = [col for col in df.columns if '아동인구' in col]
df['아동인구평균'] = df[child_cols].mean(axis=1)

# 총생활인구 평균 (22년~24년 전체 분기)
pop_cols = [col for col in df.columns if '총생활인구' in col]
df['총생활인구평균'] = df[pop_cols].mean(axis=1)

# 소득 평균 (22년~24년 전체 분기)
income_cols = [col for col in df.columns if '소득_' in col]
df['소득평균'] = df[income_cols].mean(axis=1)

# 돌봄시설_수 (키움센터)
df['돌봄시설_수'] = df['키움센터_개수']

# 대중교통_수 (버스정류장 + 지하철)
df['대중교통_수'] = df['버스정류장_수'] + df['지하철역_수']

# 유아시설_수 (유치원 + 어린이집)
df['유아시설_수'] = df['유치원_수'] + df['어린이집_수']

# 일반키즈카페_수 (그대로 사용)
df['일반키즈카페_수'] = df['일반키즈카페_개수']

# 초등학교_수 (그대로 사용)
df['초등학교_수'] = df['초등학교_수']

# 서울형 키즈카페 개수 (독립변수)
df['서울형키즈카페_개수'] = df['서울형키즈카페_개수']

# 로그 변환 수행 (생활인구, 아동인구, 소득)
df['총생활인구_log'] = np.log1p(df['총생활인구평균'])
df['아동인구_log'] = np.log1p(df['아동인구평균'])
df['소득_log'] = np.log1p(df['소득평균'])

# 석용 최종 데이터프레임 구성
selected_vars_final = [
    '행정동명', '서울형키즈카페_개수',
    '총생활인구_log', '유아시설_수', '아동인구_log',
    '일반키즈카페_수', '돌봄시설_수', '대중교통_수',
    '초등학교_수', '소득_log'
]

df_analysis = df[selected_vars_final].copy()

# Min-Max 정규화
analysis_vars = [
    '총생활인구_log', '유아시설_수', '아동인구_log',
    '일반키즈카페_수', '돌봄시설_수', '대중교통_수',
    '초등학교_수', '소득_log'
]

scaler = MinMaxScaler()
df_analysis[analysis_vars] = scaler.fit_transform(df_analysis[analysis_vars])

# 최종 정규화 가중치 설정
weights = {
    '총생활인구_log': 0.216620,
    '유아시설_수': 0.164239,
    '아동인구_log': 0.148443,
    '일반키즈카페_수': 0.102752,
    '초등학교_수': 0.089283,
    '돌봄시설_수': 0.072894,
    '대중교통_수': 0.070301,
    '소득_log': -0.076789
}

# MCDA 점수 계산
df_analysis['MCDA_점수'] = np.sum(
    [df_analysis[var] * weight for var, weight in weights.items()],
    axis=0
)

# 결과 확인 (상위 20개 행정동 추천)
result = df_analysis.sort_values(by='MCDA_점수', ascending=False)
print("서울형 키즈카페 입지추천 MCDA 점수")
print(result[['행정동명', 'MCDA_점수', '서울형키즈카페_개수']].head(20))


In [ ]:
# 위에 만든 MCDA 결과 데이터프레임 저장
result[['행정동명', 'MCDA_점수', '서울형키즈카페_개수']].to_csv(
    r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과.csv",
    encoding='utf-8-sig', index=False
)


In [ ]:
import pandas as pd
from difflib import get_close_matches

grid_df = pd.read_csv(r"C:\Users\user\Desktop\real\서울시격자_202412_보간완료.csv", encoding='utf-8')
mcda_df = pd.read_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과.csv", encoding='utf-8')

# 행정동명 유사문자열 매칭 함수 사용용
def match_name(name, name_list):
    matches = get_close_matches(name, name_list, n=1, cutoff=0.7)  
    return matches[0] if matches else None

# 격자 데이터에 유사 문자열 기반 MCDA 점수 매칭
matched_mcda_scores = []

for dong_name in grid_df['행정동명']:
    matched_name = match_name(dong_name, mcda_df['행정동명'].tolist())
    if matched_name:
        score = mcda_df.loc[mcda_df['행정동명'] == matched_name, 'MCDA_점수'].values[0]
    else:
        score = 0 
    matched_mcda_scores.append(score)

grid_df['MCDA_점수'] = matched_mcda_scores

grid_df.to_csv(r"C:\Users\user\Desktop\real\merged_result_유사매칭.csv", encoding='utf-8-sig', index=False)


In [ ]:
import pandas as pd
from difflib import get_close_matches

grid_df = pd.read_csv(r"C:\Users\user\Desktop\real\서울시격자_202412_보간완료.csv", encoding='utf-8')
mcda_df = pd.read_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과.csv", encoding='utf-8')

# 유사 매칭 결과
def match_name(name, name_list):
    matches = get_close_matches(name, name_list, n=1, cutoff=0.7)
    return matches[0] if matches else None

grid_df['matched_행정동명'] = grid_df['행정동명'].apply(lambda x: match_name(x, mcda_df['행정동명'].tolist()))

# 원본 행정동명과 매칭된 행정동명 비교 결과
comparison_df = grid_df[['행정동명', 'matched_행정동명']]

# 원본과 매칭명이 다르게 된 행만 추출하여 확인
mismatched = comparison_df[comparison_df['행정동명'] != comparison_df['matched_행정동명']]
print("매칭이 정확히 되지 않은 행정동:")
print(mismatched)

# 매칭 정확도 확인
total_count = len(grid_df)
mismatch_count = len(mismatched)
match_accuracy = (total_count - mismatch_count) / total_count * 100

print(f"총 데이터 개수: {total_count}")
print(f"매칭되지 않은 데이터 개수: {mismatch_count}")
print(f"매칭 정확도: {match_accuracy:.2f}%")


In [ ]:
import pandas as pd
from difflib import get_close_matches


mcda_df = pd.read_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과.csv", encoding='utf-8-sig')
original_df = pd.read_csv(r"C:\Users\user\Desktop\real\분석용데이터_소득생활인구_아동인구_보간완료.csv", encoding='utf-8-sig')

# 유사매칭을 통해 0점 행정동의 MCDA 점수 재매핑
def match_mcda_score(dong_name, mcda_df):
    matched_name = get_close_matches(dong_name, mcda_df['행정동명'], n=1, cutoff=0.7)
    if matched_name:
        return mcda_df.loc[mcda_df['행정동명'] == matched_name[0], 'MCDA_점수'].values[0]
    else:
        return 0 

# 현재 MCDA 점수 0인 행정동만 처리
zero_score_indices = mcda_df[mcda_df['MCDA_점수'] == 0].index

for idx in zero_score_indices:
    dong_name = mcda_df.at[idx, '행정동명']
    new_score = match_mcda_score(dong_name, mcda_df)
    mcda_df.at[idx, 'MCDA_점수'] = new_score

# 최종 보완된 파일 다시 저장 
mcda_df.to_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과_보완완료.csv", encoding='utf-8-sig', index=False)

# 결과 확인
print("0점이었던 MCDA 점수 보완 완료:")
print(mcda_df.loc[zero_score_indices, ['행정동명', 'MCDA_점수']])


In [ ]:
import pandas as pd

# 기존 geometry가 포함된 데이터
grid_df = pd.read_csv("C:\\Users\\user\\Desktop\\real\\merged_result_유사매칭.csv", encoding='utf-8-sig')

# 새 MCDA 점수 파일
mcda_df = pd.read_csv("C:\\Users\\user\\Desktop\\real\\서울형키즈카페_MCDA결과_보완완료.csv", encoding='utf-8-sig')

# '행정동명' 기준 merge
final_df = pd.merge(grid_df[['grid_id', 'geometry', '행정동명']], 
                    mcda_df[['행정동명', 'MCDA_점수', '서울형키즈카페_개수']],
                    on='행정동명', how='left')

# 저장 (geometry 추가 완료)
final_df.to_csv("C:\\Users\\user\\Desktop\\real\\서울형키즈카페_MCDA결과_보완완료_geometry.csv",
                encoding='utf-8-sig', index=False)


In [ ]:
import pandas as pd


df = pd.read_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과_보완완료_geometry.csv", encoding='utf-8-sig')

# MCDA_점수가 빈값이거나 숫자가 아닌 경우 0으로 대체
df['MCDA_점수'] = pd.to_numeric(df['MCDA_점수'], errors='coerce').fillna(0)


df.to_csv(r"C:\Users\user\Desktop\real\서울형키즈카페_MCDA결과_보완완료_geometry_최종.csv", encoding='utf-8-sig', index=False)
